# KEGG Data Processing Pipeline - Part 2: Variant Information Parsing and Sequence Generation

## Overview

This notebook is the second part of the KEGG data processing pipeline. It focuses on parsing variant information from KEGG data, generating nucleotide sequences with mutations, and creating disease mapping databases.

## What This Notebook Does

1. **Variant Information Parsing**: Extracts detailed information from KEGG variant files
2. **Sequence Generation**: Creates reference and variant nucleotide sequences with genomic context
3. **Disease Mapping**: Downloads and processes KEGG disease information
4. **Data Integration**: Merges variant data with genomic sequences and disease annotations
5. **Quality Control**: Validates reference sequences against the genome

## Prerequisites

**Required from Part 1 (KEGG_Data_1.ipynb):**
- `gene_variants.txt` - List of variant identifiers
- `variant_info/` directory - Individual variant information files
- `final_network_with_variant.tsv` - Network and variant mapping data

**Additional Requirements:**
- Reference genome FASTA file (GRCh38)
- BioPython for sequence processing
- KEGG_pull for disease information retrieval

## Required Packages

```bash
pip install biopython pandas kegg-pull
```

## Input Files Expected

- `gene_variants.txt` - Variant identifiers from Part 1
- `variant_info/*.txt` - Individual variant information files
- `chromosomes.fasta` - Reference genome sequences
- `final_network_with_variant.tsv` - Network-variant mapping

## Output Files Generated

- `nt_seq/` - Directory containing reference and variant sequences
- `verification.txt` - Quality control results
- `diseases.txt` - List of disease identifiers
- `disease_info/` - Disease information files
- Updated `final_network_with_variant.tsv` with disease names

## Important Notes

- **Memory Usage**: Processing large genomic sequences requires significant RAM
- **Storage**: Generated sequence files can be several GB in size
- **Processing Time**: Full pipeline may take several hours depending on dataset size
- **Dependencies**: Requires successful completion of KEGG_Data_1.ipynb

## Next Steps

After completing this notebook, run `KEGG_Data_3.ipynb` for final dataset creation and sequence integration.

## Configuration

Set up paths and parameters for variant processing:

In [ ]:
# Configuration - Update these paths for your environment
import os
from pathlib import Path

# Navigate to kegg_data directory
data_dir = Path('kegg_data')
if not data_dir.exists():
    print("❌ kegg_data directory not found. Please run KEGG_Data_1.ipynb first.")
    raise FileNotFoundError("kegg_data directory missing")

os.chdir(data_dir)

# Configuration parameters
CONFIG = {
    # Input files (should exist from Part 1)
    'gene_variants_file': 'gene_variants.txt',
    'variant_info_dir': 'variant_info',
    'network_data_file': 'final_network_with_variant.tsv',
    
    # Reference genome (update path as needed)
    'reference_fasta': 'chromosomes.fasta',  # Update to your reference genome path
    
    # Output directories
    'nt_seq_dir': 'nt_seq',
    'disease_info_dir': 'disease_info',
    
    # Processing parameters
    'sequence_window': 2000,  # Nucleotides around variant
    'verification_file': 'verification.txt',
    'diseases_file': 'diseases.txt'
}

# Verify required input files
required_files = ['gene_variants.txt', 'final_network_with_variant.tsv']
missing_files = []
for file in required_files:
    if not os.path.exists(file):
        missing_files.append(file)

if missing_files:
    print(f"❌ Missing required files: {missing_files}")
    print("Please run KEGG_Data_1.ipynb first to generate these files.")
else:
    print("✅ All required input files found")

# Create output directories
for dir_name in [CONFIG['nt_seq_dir'], CONFIG['disease_info_dir']]:
    Path(dir_name).mkdir(exist_ok=True)

print(f"Working directory: {os.getcwd()}")
print("\n📝 Update CONFIG['reference_fasta'] with path to your reference genome file")

In [ ]:
# Working directory already set in configuration section above
print(f"Current working directory: {os.getcwd()}")

In [3]:
sed -i '' 's/:/_/g' gene_variants.txt

In [10]:
while read p; do
    if ! grep -q NAME variant_info/$p.txt; then
        echo "$p"
    fi
done < gene_variants.txt

In [12]:
while read p; do
    if ! grep -q GENE variant_info/$p.txt; then
        echo "$p"
    fi
done < gene_variants.txt

# Pulling Info from the Variant File

# Variant Information Parsing

This section processes individual variant files to extract structured information including variant names, genes, and types.

In [ ]:
# Working directory already set - proceeding with variant information parsing
print(f"Processing variant files from: {CONFIG['variant_info_dir']}")

In [ ]:
import pandas as pd
import re
from pathlib import Path
import os

# Read all file names from gene_variants.txt
gene_variants_file = CONFIG['gene_variants_file']
if not os.path.exists(gene_variants_file):
    print(f"❌ Gene variants file not found: {gene_variants_file}")
    print("Please run KEGG_Data_1.ipynb first to generate this file")
    raise FileNotFoundError(f"Gene variants file not found: {gene_variants_file}")

with open(gene_variants_file, 'r') as f:
    variant_files = [line.strip() for line in f if line.strip()]

print(f"Processing {len(variant_files)} variant files")

# Initialize an empty DataFrame to collect the results
variant_info = pd.DataFrame(columns=["Entry", "Variant_Name", "Variant_Gene", "Variant_Gene Info", "Variant_Type"])

# Function to extract the value after a keyword (single line, rest of the line)
def extract_value(line, key):
    return line.split(key, 1)[-1].strip()

# Process each variant file
variant_info_dir = Path(CONFIG['variant_info_dir'])
processed_count = 0
not_found_count = 0

for file_name in variant_files:
    file_path = variant_info_dir / f"{file_name}.txt"

    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        name = ""
        gene = ""
        gene_info = ""
        type_info = ""

        for line in lines:
            line = line.strip()
            if line.startswith("NAME"):
                name = extract_value(line, "NAME")
            elif line.startswith("GENE"):
                gene_data = extract_value(line, "GENE")
                if gene_data:
                    parts = gene_data.split(maxsplit=1)
                    gene = parts[0]
                    gene_info = parts[1] if len(parts) > 1 else ""
            elif line.startswith("TYPE"):
                type_info = extract_value(line, "TYPE")

        row = {
            "Entry": file_name,
            "Variant_Name": name,
            "Variant_Gene": gene,
            "Variant_Gene Info": gene_info,
            "Variant_Type": type_info
        }

        variant_info = pd.concat([variant_info, pd.DataFrame([row])], ignore_index=True)
        processed_count += 1
        
        if processed_count % 100 == 0:
            print(f"Processed {processed_count}/{len(variant_files)} files...")

    except FileNotFoundError:
        print(f"[Warning] File not found: {file_path}")
        not_found_count += 1

print(f"✅ Processing complete: {processed_count} files processed, {not_found_count} files not found")
print(f"Extracted information for {len(variant_info)} variants")

# Optional: Save the final table
# variant_info.to_csv("parsed_variant_info.csv", index=False)

In [32]:
variant_info["Entry"] = variant_info["Entry"].str.replace("hsa_var_", "", regex=False)

In [33]:
variant_info

,Entry,Variant_Name,Variant_Gene,Variant_Gene Info,Variant_Type
0,1019v2,CDK4 mutation,CDK4,cyclin dependent kinase 4 [KO:K02089],
1,1027v3,CDKN1B mutation,CDKN1B,cyclin dependent kinase inhibitor 1B [KO:K06624],
2,10280v1,SIGMAR1 mutation,SIGMAR1,sigma non-opioid intracellular receptor 1 [KO:...,
3,1029v2,CDKN2A mutation,CDKN2A,cyclin dependent kinase inhibitor 2A [KO:K06621],
4,11315v1,PARK7 mutation,PARK7,Parkinsonism associated deglycase [KO:K05687],
...,...,...,...,...,...
90,9049v1,AIP mutation,AIP,AHR interacting HSP90 co-chaperone [KO:K17767],
91,9101v1,USP8 mutation,USP8,ubiquitin specific peptidase 8 [KO:K11839],
92,9217v1,VAPB mutation,VAPB,VAMP associated protein B and C [KO:K10707],
93,9817v1,KEAP1 mutation,KEAP1,kelch like ECH associated protein 1 [KO:K10456],


# Creating the Nt Variant Database

# Nucleotide Sequence Database Creation

This section creates nucleotide sequences with genomic context around each variant, generating both reference and mutated sequences for downstream analysis.

In [ ]:
# Working directory already set - proceeding with nucleotide sequence processing
print("Starting nucleotide variant database creation...")

In [2]:
from Bio import SeqIO
import pandas as pd

In [ ]:
import os
import pandas as pd

# Load network and variant data
network_file = CONFIG['network_data_file']
if not os.path.exists(network_file):
    print(f"❌ Network data file not found: {network_file}")
    print("Please run KEGG_Data_1.ipynb first to generate this file")
    raise FileNotFoundError(f"Network data not found: {network_file}")

variant_data = pd.read_csv(network_file, sep='\t')
print(f"✅ Loaded variant data: {len(variant_data)} entries")

In [4]:
len(variant_data)

1449

In [5]:
variant_data.iloc[1]["Network"]

'N00073'

In [ ]:
from Bio import SeqIO
import os

# Assuming CONFIG is defined somewhere earlier in the code
# CONFIG = {'reference_fasta': 'path_to_your_fasta_file'}

# Load reference genome sequences
fasta_file = CONFIG['reference_fasta']
if not os.path.exists(fasta_file):
    print(f"❌ Reference genome file not found: {fasta_file}")
    print("Please update CONFIG['reference_fasta'] with the correct path to your reference genome")
    print("Download from: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/")
    raise FileNotFoundError(f"Reference genome not found: {fasta_file}")

print(f"Loading reference genome from: {fasta_file}")
record_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
print(f"✅ Loaded {len(record_dict)} chromosome sequences")

In [7]:
chromosome_dictionary = {
    "1": "NC_000001.11",
    "2": "NC_000002.12",
    "3": "NC_000003.12",
    "4": "NC_000004.12",
    "5": "NC_000005.10",
    "6": "NC_000006.12",
    "7": "NC_000007.14",
    "9": "NC_000009.12",
    "10": "NC_000010.11",
    "11": "NC_000011.10",
    "12": "NC_000012.12",
    "13": "NC_000013.11",
    "14": "NC_000014.9",
    "15": "NC_000015.10",
    "16": "NC_000016.10",
    "17": "NC_000017.11",
    "18": "NC_000018.10",
    "19": "NC_000019.10",
    "20": "NC_000020.11",
    "21": "NC_000021.9",
    "23": "NC_000023.11"
}

### Verification that the reference is present at the exact position I have in my data

In [ ]:
# Verify reference sequences against genome
verification_file = CONFIG['verification_file']
print(f"Starting sequence verification - results will be saved to: {verification_file}")

with open(verification_file, "w") as f:
    for i in range(len(variant_data)):
        # ---- Input ----
        chromosome_id = chromosome_dictionary[str(variant_data.iloc[i]['Chr'])]
        if (variant_data.iloc[i]['TranscriptID'][:4] == "ENST"):
            start = variant_data.iloc[i]['Start'] - 1
        else:
            start = variant_data.iloc[i]['Start']
        reference_allele = variant_data.iloc[i]['RefAllele']
        end = len(reference_allele) + start

        chrom_seq = record_dict[chromosome_id].seq

        # Adjust for 0-based indexing in Python
        genomic_ref = chrom_seq[start: start + len(reference_allele)]

        if genomic_ref.upper() != reference_allele.upper():
            f.write(f"⚠️ Warning: Entry number {i} with variant {variant_data.iloc[i]['ID']} expected '{reference_allele}', but found '{genomic_ref}'\n")
        else:
            f.write(f"✅ Verified: {chromosome_id}:{start}-{end} → '{reference_allele}' matches genome\n")
        
        if (i + 1) % 100 == 0:
            print(f"Verified {i + 1}/{len(variant_data)} variants...")

print(f"✅ Verification complete. Results saved to: {verification_file}")

In [ ]:
from pathlib import Path

# Assuming CONFIG is defined somewhere above in the code
# CONFIG = {'nt_seq_dir': 'desired/path/to/nt_seq'}

# Create nucleotide sequence directory
nt_seq_dir = Path(CONFIG['nt_seq_dir'])
nt_seq_dir.mkdir(exist_ok=True)
print(f"Created nucleotide sequence directory: {nt_seq_dir}")

### Performing the mutation and saving the reference and variant allele with a 1000 nt window

In [ ]:
# Generate nucleotide sequences with mutations
nt_seq_dir = CONFIG['nt_seq_dir']
window = CONFIG['sequence_window']

print(f"Generating nucleotide sequences with {window}bp windows...")
print(f"Output directory: {nt_seq_dir}")

for i in range(len(variant_data)):
    output_file = f"{nt_seq_dir}/{variant_data.iloc[i]['Var_ID']}.txt"
    
    with open(output_file, "w") as f:
        # ---- Input ----
        chromosome_id = chromosome_dictionary[str(variant_data.iloc[i]['Chr'])]
        if (variant_data.iloc[i]['TranscriptID'][:4] == "ENST"):
            start = variant_data.iloc[i]['Start'] - 1
        else:
            start = variant_data.iloc[i]['Start']
        reference_allele = variant_data.iloc[i]['RefAllele']
        variant_allele = variant_data.iloc[i]['AltAllele']

        end = len(reference_allele) + start
        
        chrom_seq = record_dict[chromosome_id].seq

        # Extract region
        region_start = max(0, start - window)
        region_end = end + window

        ref_seq = chrom_seq[region_start:region_end]
    
        if (variant_allele == "deletion"):
            # Apply mutation
            mutated_seq = ref_seq[:window] + ref_seq[window + len(reference_allele):]
    
            f.write(f">{variant_data.iloc[i]['ID']}_reference_{reference_allele}\n")
            f.write(f"{ref_seq}\n")
            f.write(f">{variant_data.iloc[i]['ID']}_variant_{variant_allele}\n")
            f.write(f"{mutated_seq}\n")
        else:
            del_len = len(reference_allele)
            # Apply mutation
            mutated_seq = ref_seq[:window] + variant_allele + ref_seq[window + del_len:]
    
            f.write(f">{variant_data.iloc[i]['ID']}_reference_{reference_allele}\n")
            f.write(f"{ref_seq}\n")
            f.write(f">{variant_data.iloc[i]['ID']}_variant_{variant_allele}\n")
            f.write(f"{mutated_seq}\n")
    
    if (i + 1) % 100 == 0:
        print(f"Generated sequences for {i + 1}/{len(variant_data)} variants...")

print(f"✅ Sequence generation complete. {len(variant_data)} sequence files created in {nt_seq_dir}")

# Adding in more Variant Data

# Data Integration

This section merges variant information with the main dataset to create a comprehensive database with all relevant annotations.

In [34]:
final_data = variant_data.merge(variant_info, on='Entry')

In [ ]:
final_data

In [ ]:
# Save merged variant data
output_file = CONFIG['network_data_file']
final_data.to_csv(output_file, sep='\t', header=True, index=False)
print(f"✅ Final variant data with merged information saved to: {output_file}")
print(f"Dataset contains {len(final_data)} variants with complete information")

# Pulling Disease info

# Disease Information Processing

This section extracts disease identifiers from the variant data and downloads corresponding disease information from KEGG to create human-readable disease names.

In [44]:
import ast

In [65]:
diseases = []

In [66]:
for i in range(len(final_data)):
    diseases.extend(list(ast.literal_eval(final_data['Disease'][i]).keys()))

In [74]:
disease = set(diseases)

In [ ]:
# Save disease identifiers to file
diseases_file = CONFIG['diseases_file']
with open(diseases_file, 'w') as f:
    for disease_id in disease:
        f.write(f"{disease_id}\n")
        
print(f"✅ Saved {len(disease)} unique disease identifiers to: {diseases_file}")

In [ ]:
# Working directory already set - proceeding with disease information retrieval
print("Starting disease information processing...")

In [2]:
kegg_pull rest info disease

disease          KEGG Disease Database
ds               Release 114.0+/04-28, Apr 25
                 Kanehisa Laboratories
                 2,912 entries

linked db        pathway
                 brite
                 ko
                 hsa
                 genome
                 network
                 variant
                 drug
                 pubmed



In [ ]:
kegg_pull --full-help

In [ ]:
from pathlib import Path

# Assuming CONFIG is defined somewhere earlier in the code
# CONFIG = {'disease_info_dir': 'desired/path/to/disease_info'}

# Create disease information directory
disease_dir = Path(CONFIG['disease_info_dir'])
disease_dir.mkdir(exist_ok=True)
print(f"Created disease information directory: {disease_dir}")

In [ ]:
# Download disease information using kegg_pull
diseases_file = CONFIG['diseases_file']
disease_output_dir = CONFIG['disease_info_dir']

if not os.path.exists(diseases_file):
    print(f"❌ Diseases file not found: {diseases_file}")
    print("Please run the previous cells to generate the diseases list")
else:
    print(f"Downloading disease information for entries in: {diseases_file}")
    print(f"Output directory: {disease_output_dir}")
    # Run the command to download disease information
    !cat {diseases_file} | kegg_pull pull entry-ids - --output={disease_output_dir}
    print("✅ Disease information download complete")

100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.56it/s]


In [ ]:
# Processing disease information files
print("Parsing disease information from KEGG files...")

In [ ]:
# Parse disease information from downloaded files
diseases_file = CONFIG['diseases_file']
disease_info_dir = Path(CONFIG['disease_info_dir'])

# Read all disease identifiers from diseases.txt
with open(diseases_file, 'r') as f:
    disease_files = [line.strip() for line in f if line.strip()]

print(f"Processing {len(disease_files)} disease information files...")

# Initialize an empty dictionary
disease_info = {}

# Function to extract the value after a keyword
def extract_value(line, key):
    return line.split(key, 1)[-1].strip()

# Process each disease file
processed_count = 0
not_found_count = 0

for disease_id in disease_files:
    file_path = disease_info_dir / f'{disease_id}.txt'

    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        name = ""

        for line in lines:
            line = line.strip()
            if line.startswith("NAME"):
                name = extract_value(line, "NAME")
                break  # No need to check other lines once NAME is found

        # Save into dictionary: key = disease_id, value = name
        disease_info[disease_id] = name
        processed_count += 1
        
        if processed_count % 50 == 0:
            print(f"Processed {processed_count}/{len(disease_files)} disease files...")

    except FileNotFoundError:
        print(f"[Warning] File not found: {file_path}")
        not_found_count += 1

print(f"✅ Disease processing complete: {processed_count} processed, {not_found_count} not found")
print(f"Extracted disease information for {len(disease_info)} diseases")

# Optional: Save the dictionary to a file (like JSON)
# import json
# with open('disease_info.json', 'w') as f:
#     json.dump(disease_info, f, indent=2)

In [8]:
disease_info

{'H00135': 'Krabbe disease;',
 'H01398': 'Primary hyperammonemia (Urea cycle disorders)',
 'H00032': 'Thyroid cancer',
 'H00559': 'von Hippel-Lindau syndrome',
 'H00260': 'Pigmented micronodular adrenocortical disease',
 'H00038': 'Melanoma',
 'H00485': 'Robinow syndrome',
 'H00251': 'Thyroid dyshormonogenesis;',
 'H00194': 'Lesch-Nyhan syndrome;',
 'H00026': 'Endometrial cancer',
 'H00020': 'Colorectal cancer',
 'H00031': 'Breast cancer',
 'H02049': 'Bilateral macronodular adrenal hyperplasia',
 'H00042': 'Glioma',
 'H00063': 'Spinocerebellar ataxia (SCA)',
 'H00195': 'Adenine phosphoribosyltransferase deficiency;',
 'H00033': 'Adrenal carcinoma',
 'H00048': 'Hepatocellular carcinoma;',
 'H01522': 'Zollinger-Ellison syndrome',
 'H00019': 'Pancreatic cancer',
 'H00004': 'Chronic myeloid leukemia',
 'H00058': 'Amyotrophic lateral sclerosis (ALS);',
 'H00022': 'Bladder cancer',
 'H00056': 'Alzheimer disease;',
 'H01032': 'N-acetylglutamate synthase deficiency',
 'H00247': 'Multiple endoc

In [ ]:
# Reload variant data for disease processing
variant_data = pd.read_csv(CONFIG['network_data_file'], sep='\t')
print(f"Processing disease information for {len(variant_data)} variants")

In [10]:
import ast

# Assume disease_info is already a dictionary {"D001": "Cancer", "D002": "Diabetes", ...}

# Create a new column to store disease dictionaries
variant_data["Disease_Names"] = ""

# Process each row
for idx, row in variant_data.iterrows():
    try:
        # Convert the string dictionary into a real dictionary
        disease_dict = ast.literal_eval(row["Disease"])

        # Get the disease IDs (keys)
        disease_ids = disease_dict.keys()

        # Build a new dictionary: {disease_id: disease_name}
        disease_names_dict = {did: disease_info.get(did, "") for did in disease_ids}

        # Save it into the Disease_Names column
        variant_data.at[idx, "Disease_Names"] = disease_names_dict

    except (ValueError, SyntaxError):
        print(f"[Warning] Couldn't parse disease info at row {idx}")
        variant_data.at[idx, "Disease_Names"] = {}

In [ ]:
# Save updated variant data with disease names
output_file = CONFIG['network_data_file']
variant_data.to_csv(output_file, sep='\t', header=True, index=False)
print(f"✅ Updated variant data saved to: {output_file}")
print(f"Dataset now includes disease names for {len(variant_data)} variants")